In [2]:
import ipywidgets
from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual, HBox, fixed
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
layout = widgets.Layout(width='400px', height='30px')
# HIDING All the snippets
import pandas as pd
import matplotlib.pyplot as plt
# import statsmodels.graphics.tsaplots as sgt
# import statsmodels.tsa.stattools as sts
# from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
# import seaborn as sns
# import pylab
# import scipy.stats
#sns.set()
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
from plotly.offline import iplot
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import plotly.express as px
import plotly.graph_objects as go
import json
from urllib.request import urlopen
from fbprophet import Prophet


In [3]:
#Extracting and Preprocessing Data from Covid-19 API
df = pd.read_csv('https://api.covid19india.org/csv/latest/states.csv')
df = df[df.State != 'India']
df = df[df.State != 'State Unassigned']
df.drop(['Other','Tested'],axis=1,inplace=True)
df.rename(columns={'Recovered': 'Discharged','Deceased': 'Death'},inplace = True)
df['Active'] = df.apply(lambda row: row.Confirmed - (row.Discharged + row.Death), axis = 1)
df_o = df.copy()
df.Date = pd.to_datetime(df.Date)
df.set_index("Date", inplace = True)
df = df[['State','Confirmed','Death','Active','Discharged']]
#Classifying The DataFrame to distribute the data among states of India
states = df.groupby('State')
state={}
l=[]
for i,j in states:
    state[i] = j[-365:]
for i,j in state.items():
    l.append(i)
del states
states_list = l

# Setting dialy frequency State-wise and filling the Data for the dates which are not recorded(missed), with previous date data. 
for i in state.keys():
    tdf = state[i]
    tdf = tdf[['Confirmed','Death','Active','Discharged']].astype(int)
    tdf = tdf.asfreq('d')
    state[i] = tdf.fillna(method = 'ffill')
min_date = state['Goa'].index.min()
max_date = state['Goa'].index.max()
pdf = df_o.copy()
pdf = pdf[['Date','State','Confirmed','Death','Active','Discharged']]
#Classifying The DataFrame to distribute the data among states of India
pstates = pdf.groupby('State')
pstate={}
for i,j in pstates:
    pstate[i] = j[-365:]


In [4]:
#Preparng Dataframes for forecasting data of next 30 days

fut_state={}

for i,j in pstate.items():
    fd = pd.DataFrame(columns=['Date','Confirmed', 'Death', 'Discharged'])
    for k in ['Confirmed','Death','Discharged']:
        ts_d = j[['Date',k]][-50:].copy()
        ts_d.rename(columns={'Date':'ds',k:'y'}, inplace=True)
        m= Prophet(interval_width = 0.95,daily_seasonality = True, yearly_seasonality = False)
        m.fit(ts_d) 
        future = m.make_future_dataframe(periods=30) 
        forecast = m.predict(future)
        fd['Date']=forecast.ds[-30:]
        fd[k]=forecast.yhat[-30:]
        del m
    j = j.append(fd,ignore_index = True)
    j.Date = pd.to_datetime(j.Date)
    j.set_index("Date", inplace = True)
    j = j[['Confirmed','Death','Discharged']].astype(int)
    j = j[['Confirmed','Death','Discharged']]
    fut_state[i] = j


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [5]:
#Prediction Analysis
select_state = widgets.Dropdown(
    options=states_list,
    value='Telangana',
    description='Select State:',
    disabled=False,
    )    
def plotg(select_state):
    print('Showing Scatter PLOT for the coming 30 Days of ',select_state.upper(),'State',' (predicted using FBPROPHET)')
    fut_state[select_state].iplot(xTitle='<----  Date  ---->', yTitle='<----  Number of Cases  ---->',
                 title = select_state, width=5,dash=['dash','dot','dashdot'])
    
#interact(plotg,select_state=select_state)

In [6]:
#Daily Analysis
#Creating widgets and Dahboard Functions
#Extracting Geojson file for states of India
with urlopen('https://raw.githubusercontent.com/sharif9911/Interactive-Covid-19-India--Dashboard-with-in-Jupyter-Notebook/main/states_india.geojson') as response:
    counties = json.load(response)
select_date = df.index.max()
select_date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
)
kinda_cat=widgets.ToggleButtons(
    options=['Confirmed', 'Active', 'Death', 'Discharged'],
    description='Category:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    #     tooltips=['Sc', 'Bar', 'Box],
    #icons=['check'] * 3
    )

view=widgets.ToggleButtons(
    options=['BAR view','MAP view'],
    description='Plot Type:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #     tooltips=['Sc', 'Bar', 'Box],
    #icons=['check'] * 3
    )
def day_bar(select_date,view):
    if(select_date == None):
        select_date= max_date
    if(select_date < df.index.min() or select_date > df.index.max()):
        print('PLEASE PICK A VALID DATE IN THE CALENDAR i.e From   ',str(min_date)[:10],'  to  ' ,str(max_date)[:10],'\n ----Refer to the Database for more Information!')
        return
    t_df = pd.DataFrame(columns=['Confirmed', 'Active', 'Death', 'Discharged'])
    for i,j in state.items():
        t_df.loc[i]= j.loc[str(select_date)]
        t_df = t_df[['Confirmed','Death','Active','Discharged']].astype(int)
    theme= {'Discharged' : 'sunset', 'Active' : 'picnic', 'Death' : 'reds', 'Confirmed': 'turbid'}
    def plot_bar(kinda_cat):
        if view =='BAR view':
            fig = px.bar(t_df,x=t_df.index, y=kinda_cat, text=kinda_cat, color=t_df.index,title= kinda_cat + ' Cases on '+ str(select_date)[:10],
                labels={'index': 'State', 'Active': str(kinda_cat + ' cases')},width=1400,height=900)

            fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
            fig.update_layout(uniformtext_minsize=8, xaxis_tickangle=-45)
            fig.show()
        else:
            fig = px.choropleth(t_df, geojson=counties, locations=t_df.index, color=kinda_cat,
                           color_continuous_scale=theme[kinda_cat],
                           featureidkey='properties.ST_NM',
                        labels={'index':'State'}, title='Map Showing '+kinda_cat+' Cases on '+str(select_date)[:10]
                          )
            fig.update_geos(fitbounds="locations", visible=False,  showsubunits=True, subunitcolor="Green")
            fig.show()
            
    interact(plot_bar, kinda_cat=kinda_cat)
# interact(day_bar, select_date=select_date,view = view)

In [7]:
#State-wise Analysis
def stategraph():
    select_state = widgets.Dropdown(
    options=states_list,
    value='Telangana',
    description='Select State:',
    disabled=False,
    )
    
    kinda_plot=widgets.ToggleButtons(
    options=['Scatter Plot', 'Box Plot', 'Bar Plot','Pie Plot'],
    description='Plot Type:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    #     tooltips=['Sc', 'Bar', 'Box],
    #icons=['check'] * 3
    )
    
    
    def printgraph(select_state,kinda_plot):
        date_range = widgets.IntRangeSlider(
        value=[0, state[select_state].count().max()],
        min=0,
        max=state[select_state].count().max()-1,
        step=1,
        disabled=False,
        description='Period:',
        continuous_update=False,
        layout=widgets.Layout(width='90%'),
        orientation='horizontal',
        readout=False,
        readout_format='d'
        )
        def plot(date_range):
            if kinda_plot != 'Pie Plot':
                print('Showing',kinda_plot,'from','\t',str(state[select_state].index[date_range[0]])[:10],
                  '\t\t','TO','\t\t',str(state[select_state].index[date_range[1]])[:10])
                state[select_state].iloc[date_range[0]:date_range[1]].iplot(kind = kinda_plot.split()[0].lower(),
                xTitle='<----  Date  ---->', yTitle='<----  Number of Cases  ---->',
                title = select_state, width=5,dash=['solid','dash','dot','dashdot'],
                )
            else:
                print('!! Please move the Right Extreme pointer to change period !!\n')
                print('Showing',kinda_plot,'on',str(state[select_state].index[  max(date_range[0],date_range[1]) ])[:10],' for '+select_state.upper())
                colors = ['blue', 'black', 'red', 'green']
                fig = go.Figure(data=[go.Pie(labels=['Confirmed','Death','Active','Discharged'], 
                       values=state[select_state].iloc[ max(date_range[0],date_range[1]) ])])
                # Define hover info, text size, pull amount for each pie slice, and stroke
                fig.update_traces(hoverinfo='label+value', textfont_size=20,
                  textinfo='label+percent', pull=[0, 0, 0.1, 0.1],
                  marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2)))
                fig.show()
            
        interact(plot,date_range=date_range)
    interact(printgraph,select_state=select_state,kinda_plot=kinda_plot)
    
#interact(stategraph)

![Image of Institution](http://www.griet.ac.in/images/GRIET_Logo.png)

## Dashboard for Analysis of Covid-19 cases until `Yesterday*

In [8]:
#Main Interface of Dashboard
select_analysis = widgets.ToggleButtons(
    options=['Past Analysis', 'Prediction Analysis'],
    description='Analysis:',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    #     tooltips=['Sc', 'Bar', 'Box],
    #icons=['check'] * 3
    )
def analy(select_analysis):
    if(select_analysis[1] == 'a'):
        past = widgets.ToggleButtons(
        options=['State-wise Analysis', 'Daily Analysis'],
        description='Type:',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        #     tooltips=['Sc', 'Bar', 'Box],
        #icons=['check'] * 3
        )
        
        def p_analy(past):
            if past[0]=='S':
                interact(stategraph)
            else:
                interact(day_bar, select_date=select_date,view = view)
        interact(p_analy,past=past)
    else:
        interact(plotg,select_state=select_state)
interact(analy,select_analysis=select_analysis)

interactive(children=(ToggleButtons(button_style='warning', description='Analysis:', options=('Past Analysis',…

<function __main__.analy(select_analysis)>

In [9]:
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle user/developer mode."></form>''')